<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.activations import relu, elu, sigmoid, linear
from tensorflow.keras.optimizers import Adam, Nadam, SGD
from tensorflow.keras.losses import binary_crossentropy, hinge, squared_hinge
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [3]:
df = pd.read_csv('https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv', header=0)
df = df.drop(columns=['customerID', 'gender'])

In [4]:
enc = OrdinalEncoder()
scaler = StandardScaler()

In [5]:
features = df.columns[:-1]
target = 'Churn'
df[features] = enc.fit_transform(df[features])
df[target] = df[target].replace('No', 0).replace('Yes', 1)
train, val = train_test_split(df, test_size=.2)
X_train, y_train = train[features], train[target]
X_val, y_val = val[features], val[target]
X_train = enc.fit_transform(X_train)
X_train = scaler.fit_transform(X_train)
X_val = enc.fit_transform(X_val)
X_val = scaler.fit_transform(X_val)

In [6]:
first_neuron = [12, 18, 24, 48, 96]
hidden_layer = [6, 9, 12, 24, 48]
activation = ['relu']
optimizer = [Adam]
loss_function = [binary_crossentropy, hinge, squared_hinge]
batch_size = [2, 4]
epochs = [50, 100, 150]
lr = [1, 10, 20]
dropout = [.1, .2, .4]
last_activation = [sigmoid]
weight_initializer = ['glorot_uniform']

params = dict(first_neuron=first_neuron, hidden_layer=hidden_layer, activation=activation,
              optimizer=optimizer, lr=lr, dropout=dropout, last_activation=last_activation,
              weight_initializer=weight_initializer, epochs=epochs, batch_size=batch_size,
              loss_function=loss_function)

In [7]:
def telecom(first_neuron=18, hidden_layer=10, activation='relu',
            optimizer=Adam, lr=lr, dropout=0, last_activation=sigmoid,
            weight_initializer='uniform', epochs=100, batch_size=100,
            loss_function=binary_crossentropy):
    
    model = Sequential()

    model.add(Dense(units=first_neuron, 
                    input_dim=18, activation=activation,
                    kernel_initializer=weight_initializer
                    ))
  
    model.add(Dense(units=hidden_layer, 
                    activation=activation))

    model.add(Dense(1, activation=last_activation))

    model.compile(loss=loss_function, 
                  optimizer=Adam
                  (lr=lr), 
                  metrics=['accuracy'])
  
    return model
  
model = KerasClassifier(build_fn=telecom)

In [8]:
search = RandomizedSearchCV(model,
                      params,
                      scoring='accuracy',
                      n_jobs=-2,
                      n_iter=50,
                      cv=3,
                      verbose=1)

search.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.


KeyboardInterrupt: 

In [49]:
search.best_params_

{'weight_initializer': 'glorot_uniform',
 'optimizer': tensorflow.python.keras.optimizer_v2.adam.Adam,
 'lr': 10,
 'loss_function': <function tensorflow.python.keras.losses.hinge(y_true, y_pred)>,
 'last_activation': <function tensorflow.python.keras.activations.sigmoid(x)>,
 'hidden_layer': 24,
 'first_neuron': 48,
 'epochs': 150,
 'dropout': 0.1,
 'batch_size': 2,
 'activation': 'relu'}

In [50]:
search.best_score_

0.7362442314518992

In [69]:
search.score(X_val, y_val)

0.7338537970191625

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout

stop = EarlyStopping(monitor='val_accuracy', min_delta=.01, patience=5)

def telecom_final(X_train, y_train):
    
    model = Sequential()
    
    model.add(Dense(units=18, 
                    input_dim=18, activation='relu',
                    kernel_initializer='glorot_uniform'
                    ))
    
    model.add(Dense(units=10, 
                    activation='relu'))
    
    model.add(Dropout(.2))
    model.add(Dense(1, activation=sigmoid))

    model.compile(loss=hinge, 
                  optimizer=Adam
                  (lr=.1), 
                  metrics=['accuracy'])
    
    out = model.fit(X_train, y_train, epochs=999, batch_size=32, validation_split=.2, callbacks=[stop])
  
    return model.evaluate(X_val, y_val)

In [ ]:
telecom_final(X_train, y_train)

Train on 4507 samples, validate on 1127 samples
Epoch 1/999
4507/4507 [==============================] - 5s 1ms/sample - loss: 1.0002 - accuracy: 0.7273 - val_loss: 1.0000 - val_accuracy: 0.7471
Epoch 2/999
4507/4507 [==============================] - 5s 1ms/sample - loss: 1.0000 - accuracy: 0.7273 - val_loss: 1.0000 - val_accuracy: 0.7471
Epoch 3/999
4507/4507 [==============================] - 4s 996us/sample - loss: 1.0000 - accuracy: 0.7273 - val_loss: 1.0000 - val_accuracy: 0.7471
Epoch 4/999
4507/4507 [==============================] - 5s 1ms/sample - loss: 1.0000 - accuracy: 0.7273 - val_loss: 1.0000 - val_accuracy: 0.7471
Epoch 5/999
1368/4507 [========>.....................] - ETA: 2s - loss: 1.0000 - accuracy: 0.7127

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?